In [ ]:
#the system instruction our vertex ai is working on.
s1 = {
  "system_instruction": {
    "role": "legal document assistant",
    "content": [
      "You are a highly reliable legal document assistant specialized in answering questions about legal documents.",
      "Your role is to extract, summarize, and clarify content from provided documents without altering meaning.",
      "Always ground your responses in the text of the document. If the document does not contain the answer, state clearly that the information is not available.",
      "Do not provide legal advice or personal opinions. Remain neutral and factual.",
      "When referencing information, cite the relevant section, clause, or paragraph from the document whenever possible.",
      "Use plain and concise language, but preserve legal precision.",
      "If a question is ambiguous, ask for clarification rather than assuming."
    ]
  }
}

'''
you are a highly qualified legal professional, renowned for your sharp wit, unparalleled expertise, and ability to win even the toughest cases. As a top-tier legal advisor and document assistant, you are well-versed in all areas of law, including corporate, criminal, civil, tax, intellectual property, international, and regulatory law in the Indian jurisdiction specifically. You provide precise, actionable legal advice, identifying legitimate strategies, exemptions, or loopholes to minimize penalties or liabilities when requested, without ever endorsing illegal actions.
'''


'\nyou are a highly qualified legal professional, renowned for your sharp wit, unparalleled expertise, and ability to win even the toughest cases. As a top-tier legal advisor and document assistant, you are well-versed in all areas of law, including corporate, criminal, civil, tax, intellectual property, international, and regulatory law in the Indian jurisdiction specifically. You provide precise, actionable legal advice, identifying legitimate strategies, exemptions, or loopholes to minimize penalties or liabilities when requested, without ever endorsing illegal actions.\n'

In [ ]:
pip install google-cloud-storage
pip install google-cloud-aiplatform
pip install google-generativeai


In [1]:
#this is how we are going to fetch the files, this is how the workflow is going to be, uploaded file will be sent to our bucket.
#and the GS util URI will be returned which can be sent to the Vertex AI API for processing.

from google.cloud import storage

def upload_blob_and_get_uri(bucket_name, source_file_name, destination_blob_name, project_id):
    """Uploads a file to the specified bucket and returns its gs:// URI."""
    # Pass the project_id explicitly
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    gs_uri = f"gs://{bucket_name}/{destination_blob_name}"

    print(f"File {source_file_name} uploaded to {destination_blob_name} in bucket {bucket_name}.")
    print(f"GS URI: {gs_uri}")

    return gs_uri


In [2]:

your_bucket_name = "legal-doc-bucket1"  # Replace with your actual bucket name
local_file_path = r"C:\\Users\\user\\OneDrive\\Documents\\GenAI-exchange\\backend\\pdf\\generative-ai_pdf_certificate_of_incoporation.pdf" # Replace with the path to the file you want to upload
destination_object_name = "generative-ai_pdf_certificate_of_incoporation.pdf" # The name you want the file to have in the bucket
project_id = "sodium-coil-470706-f4" # Use your actual project ID

# Call the function to upload and get the URI
uploaded_uri = upload_blob_and_get_uri(
    your_bucket_name,
    local_file_path,
    destination_object_name,
    project_id
)

if uploaded_uri:
    print(f"\nSuccessfully obtained GS URI: {uploaded_uri}")


c:\Users\user\OneDrive\Documents\GenAI-exchange\backend\venv\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


File C:\\Users\\user\\OneDrive\\Documents\\GenAI-exchange\\backend\\pdf\\generative-ai_pdf_certificate_of_incoporation.pdf uploaded to generative-ai_pdf_certificate_of_incoporation.pdf in bucket legal-doc-bucket1.
GS URI: gs://legal-doc-bucket1/generative-ai_pdf_certificate_of_incoporation.pdf

Successfully obtained GS URI: gs://legal-doc-bucket1/generative-ai_pdf_certificate_of_incoporation.pdf


In [3]:
#the bot has been implemented below.
from google import genai
from google.genai import types
import base64

def generate():
  client = genai.Client(
      vertexai=True,
      project="sodium-coil-470706-f4",
      location="global", 
  )

  document1 = types.Part.from_uri(
      file_uri=uploaded_uri, #the uploaded uri we fetched from the above cell would be passed here.
      mime_type="application/pdf",
  )
  
  text1 = types.Part.from_text(text="""What's the exact date that the certificate of incorporation / authority be revoked? Explain your logic.""")
  si_text1 = """\"You are a highly reliable legal document assistant specialized in answering questions about legal documents.\",
      \"Your role is to extract, summarize, and clarify content from provided documents without altering meaning.\",
      \"Always ground your responses in the text of the document. If the document does not contain the answer, state clearly that the information is not available.\",
      \"Do not provide legal advice or personal opinions. Remain neutral and factual.\",
      \"When referencing information, cite the relevant section, clause, or paragraph from the document whenever possible.\",
      \"Use plain and concise language, but preserve legal precision.\",
      \"If a question is ambiguous, ask for clarification rather than assuming.\""""

  model = "gemini-2.5-flash-lite"
  contents = [
    types.Content(
      role="user",
      parts=[
        document1,
        text1
      ]
    )
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 65535,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    system_instruction=[types.Part.from_text(text=si_text1)],
    thinking_config=types.ThinkingConfig(
      thinking_budget=0,
    ),
  )

  for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    print(chunk.text, end="")

generate()

The document states that the Certificate of Incorporation/Authority will be revoked after 60 days from the date of the notice. The notice is dated June 19, 2023. Therefore, the certificate of incorporation/authority will be revoked on August 18, 2023.

In [ ]:
from google import genai
from google.genai import types

def generate_conversation():
    # Initialize client (Vertex AI or Gemini API)
    client = genai.Client(
      vertexai=True,
      project="sodium-coil-470706-f4",
      location="global", 
    )

    system_instruction_text = """
    You are a highly reliable legal document assistant specialized in answering questions about legal documents.
    Your role is to extract, summarize, and clarify content from provided documents without altering meaning.
    Always ground your responses in the text of the document. If the document does not contain the answer, state clearly that the information is not available.
    Do not provide legal advice or personal opinions. Remain neutral and factual.
    When referencing information, cite the relevant section, clause, or paragraph from the document whenever possible.
    Use plain and concise language, but preserve legal precision.
    If a question is ambiguous, ask for clarification rather than assuming.
    """

    # Create a chat session
    chat = client.chats.create(
        model="gemini-2.5-flash-lite",
        config=types.GenerateContentConfig(
            system_instruction=system_instruction_text,
            temperature=1,
            top_p=0.95,
            max_output_tokens=65535,
            safety_settings=[
                types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),     
                types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
                types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
                types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
            ],
        ),
    )

    # Load the document (from GCS bucket)
    document_part = types.Part.from_uri(
        file_uri="gs://your-bucket-name/your-document.pdf",
        mime_type="application/pdf",
    )

    # First user question
    initial_question = "What's the exact date that the certificate of incorporation / authority be revoked? Explain your logic."

    print("User:", initial_question)

    response = chat.send_message(
        message=[document_part, types.Part.from_text(text=initial_question)]
    )
    print("Bot:", response.text)

    # Loop for follow-ups
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Ending conversation.")
            break

        response = chat.send_message(user_input)
        print("Bot:", response.text)


# Run it
generate_conversation()

User: What's the exact date that the certificate of incorporation / authority be revoked? Explain your logic.


ClientError: 403 PERMISSION_DENIED. {'error': {'code': 403, 'message': 'The billing account for the owning project is disabled in state closed', 'status': 'PERMISSION_DENIED'}}

In [11]:
# Print the version of all the libraries being used

import flask
import google.cloud.storage
import google.cloud.aiplatform
import google.generativeai

print("flask version:", flask.__version__)
print("google-cloud-storage version:", google.cloud.storage.__version__)
print("google-cloud-aiplatform version:", google.cloud.aiplatform.__version__)
print("google-generativeai version:", google.generativeai.__version__)


flask version: 3.1.2
google-cloud-storage version: 2.19.0
google-cloud-aiplatform version: 1.111.0
google-generativeai version: 0.8.5


C:\Users\user\AppData\Local\Temp\ipykernel_30648\2904344997.py:8: DeprecationWarning: The '__version__' attribute is deprecated and will be removed in Flask 3.2. Use feature detection or 'importlib.metadata.version("flask")' instead.
  print("flask version:", flask.__version__)
